# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [16]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [17]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra
['/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-09-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-04-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-01-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-10-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-06-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-03-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-08-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-05-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-02-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-07-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [18]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['Muse', 'Logged In', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '200', '1.54172E+12', '42']
['Beastie Boys', 'Logged In', 'Harper', 'M', '2', 'Barrett', '161.56689', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Lighten Up', '200', '1.54172E+12', '42']
['Shakira', 'Logged In', 'Harper', 'M', '3', 'Barrett', '145.84118', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Pienso En Ti', '200', '1.54172E+12', '42']
['Selena', 'Logged In', 'Harper', 'M', '4', 'Barrett', '172.66893', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Amor Prohibido', '200', '1.54172E+12', '42']
['Kid Cudi Vs Crookers', 'Logged In', 'Harper', 'M', '5', 'Barrett', '162.97751', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSon

['', 'Logged In', 'Anabelle', 'F', '2', 'Simpson', '', 'free', 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', 'GET', 'Home', '1.54104E+12', '259', '', '200', '1.54139E+12', '69']
['Euge Groove', 'Logged In', 'Jahiem', 'M', '0', 'Miles', '265.482', 'free', 'San Antonio-New Braunfels, TX', 'PUT', 'NextSong', '1.54082E+12', '245', 'Tenderly', '200', '1.54139E+12', '43']
['Metallica / Marianne Faithfull', 'Logged In', 'Katherine', 'F', '0', 'Gay', '279.11791', 'free', 'San Antonio-New Braunfels, TX', 'PUT', 'NextSong', '1.54097E+12', '56', 'The Memory Remains', '200', '1.5414E+12', '57']
['Nana Caymmi', 'Logged In', 'Katherine', 'F', '1', 'Gay', '174.41914', 'free', 'San Antonio-New Braunfels, TX', 'PUT', 'NextSong', '1.54097E+12', '56', 'Fim De Caso', '200', '1.5414E+12', '57']
['Johnny Burnette', 'Logged In', 'Katherine', 'F', '2', 'Gay', '134.84363', 'free', 'San Antonio-New Braunfels, TX', 'PUT', 'NextSong', '1.54097E+12', '56', "I've Got A Lot Of Things To Do", '200', '1.5414E+12', '5

In [19]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1595


# Part II. Apache Cassandra 

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [20]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

try:
    from cassandra.cluster import Cluster
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [21]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [22]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Excpetion as e:
    print(e)

## Created queries to ask the following three questions of the data:

### 1. Output  the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [23]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS session "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify that the data have been inserted into each table

In [25]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, length FROM session WHERE sessionId=338 AND iteminsession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### 2. Output name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [47]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

In [48]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName FROM users WHERE userId=10 AND sessionId=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.firstName, row.lastname, row.song)

### 3. Output every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

### Drop the tables before closing out the sessions

In [55]:
## TO-DO: Drop the table before closing out the sessions
session.execute("drop table if exists session")
session.execute("drop table if exists user_session")

### Close the session and cluster connection¶

In [56]:
session.shutdown()
cluster.shutdown()